# Progetto Applicazioni Data Intensive 2023
#### Autori: Alessandro Sciarrillo e Niccolò Mussoni

In [106]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

# import necessari
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

In [107]:
#URL = "https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction"
#data_test = pd.read_csv("test.csv", sep=",")
#data_train = pd.read_csv("train.csv", sep=",")
#frames = [data_test, data_train]
#ds = pd.concat(frames)

dataset = pd.read_csv("../../Downloads/MetroPT3(AirCompressor).csv", sep=",", index_col="timestamp", parse_dates=["timestamp"])

In [108]:
# dall'1/02/2020 al 1/09/2020
dataset.head(2)
dataset.tail(2)

,Unnamed: 0,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
timestamp,,,,,,,,,,,,,,,,
2020-09-01 03:59:40,15169460,-0.012,8.876,8.864,-0.022,8.878,59.550,0.0450,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
2020-09-01 03:59:50,15169470,-0.014,8.860,8.848,-0.022,8.864,59.475,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0


In [130]:
# x=H1  y=COMP

def split_crashDay(X, y):
    #is_train = X.index.year < 2005
    #is_train = X.index.month == 12 #and X.index.day == 4 and X.index.year == 2020
    is_train = X.index.date != datetime.date(2020, 4, 12)
    #is_train = ~is_train
    
    X_train = X.loc[is_train]
    y_train = y.loc[is_train]
    X_val = X.loc[~is_train]
    y_val = y.loc[~is_train]
    return X_train, X_val, y_train, y_val

#timestamp = dataset["timestamp"]
h1 = dataset["H1"]
comp = dataset["COMP"]

features = {"H1": h1}
X = pd.DataFrame(features)
y = comp

X_train, X_val, y_train, y_val = split_crashDay(X, y)

X_val.head(5)

,H1
timestamp,
2020-04-12 00:00:01,8.808
2020-04-12 00:00:11,8.794
2020-04-12 00:00:21,8.782
2020-04-12 00:00:31,8.768
2020-04-12 00:00:40,8.756


In [131]:
# Modello casuale
np.random.seed(42)
random_preds = np.random.normal(
    y_train.mean(),   # centro (media), circa 0
    y_train.std(),    # scala (dev. standard)
    len(y_val)        # numero di campioni
)

print(len(y_val))
(random_preds-y_val).pow(2).sum()

7125


4147.020553299068

In [132]:
model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_val, y_val)

0.9924892105292286

In [133]:
predicts = model.predict(X_val)
(predicts-y_val).pow(2).sum()

12.344381489432465

In [134]:
model = Pipeline([
    ("poly",    PolynomialFeatures(degree=10, include_bias=False) ),
    ("regr", Ridge(alpha=1.0) )
])

model.fit(X_train, y_train)


Pipeline(steps=[('poly', PolynomialFeatures(degree=10, include_bias=False)),
                ('regr', Ridge())])

In [135]:
model.score(X_val, y_val)

0.9996506716113364

In [136]:
predicts = model.predict(X_val)
(predicts-y_val).pow(2).sum()

0.5741397640733581

In [137]:
model = Pipeline([
    ("poly",    PolynomialFeatures(degree=3, include_bias=False) ),
    ("norm", StandardScaler() ),
    ("regr", LinearRegression() )
])

model.fit(X_train, y_train)

Pipeline(steps=[('poly', PolynomialFeatures(degree=3, include_bias=False)),
                ('norm', StandardScaler()), ('regr', LinearRegression())])

In [138]:
model.score(X_val, y_val)

0.9995987224740824

In [139]:
predicts = model.predict(X_val)
(predicts-y_val).pow(2).sum()

0.6595209308340878

## Contesto e Obbiettivo del modello
(Linee guida: Descrivere in modo chiaro il contesto e l’obiettivo del modello di predizione, la fonte e la struttura del
dataset utilizzato e le variabili che contiene. Eseguire eventualmente una prima scrematura dei dati,
eliminando ad es. variabili palesemente non informative (es. identificatori) o con molti dati nulli.)

## Analisi Esplorativa
(Linee guida: Eseguire un’analisi esplorativa del dataset, riportando statistiche generali (medie, quartili, valori distinti,
indici di correlazione, …) e distribuzioni delle variabili in tabelle e grafici (a torta, istogrammi, a dispersione,
…). Commentare adeguatamente i risultati dell’analisi ed utilizzarli eventualmente per eliminare dati non
utilizzabili nell’analisi.)

In [6]:
dataset.describe()

,Unnamed: 0,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
count,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06
mean,7.584735e+06,1.367826e+00,8.984611e+00,7.568155e+00,5.595619e-02,8.985233e+00,6.264418e+01,2.050171e+00,8.369568e-01,1.606106e-01,9.198483e-01,8.326640e-01,3.420025e-03,9.914368e-01,9.041556e-01,9.371066e-01
std,4.379053e+06,3.250930e+00,6.390951e-01,3.333200e+00,3.824015e-01,6.383070e-01,6.516261e+00,2.302053e+00,3.694052e-01,3.671716e-01,2.715280e-01,3.732757e-01,5.838091e-02,9.214078e-02,2.943779e-01,2.427712e-01
min,0.000000e+00,-3.200000e-02,7.300000e-01,-3.600000e-02,-3.200000e-02,7.120000e-01,1.540000e+01,2.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.792368e+06,-1.400000e-02,8.492000e+00,8.254000e+00,-2.200000e-02,8.494000e+00,5.777500e+01,4.000000e-02,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
50%,7.584735e+06,-1.200000e-02,8.960000e+00,8.784000e+00,-2.000000e-02,8.960000e+00,6.270000e+01,4.500000e-02,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
75%,1.137710e+07,-1.000000e-02,9.492000e+00,9.374000e+00,-1.800000e-02,9.492000e+00,6.725000e+01,3.807500e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
max,1.516947e+07,1.067600e+01,1.030200e+01,1.028800e+01,9.844000e+00,1.030000e+01,8.905000e+01,9.295000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


## Preparazione Dati
(Linee guida: Preparare i dati per l’addestramento e la validazione dei modelli di predizione: isolare la variabile “target”
da predire e le variabili predittive, suddividere i dati in training e test set, eseguire eventuali operazioni di
preprocessing come ad es. one-hot encoding di variabili categoriche e oversampling o undersampling in
caso di classi sbilanciate.)

## Addestramento e Validazione
(Linee guida: Addestrare e validare due o più modelli di predizione, calcolandone le misure di performance viste nel
corso (es. MSE, errore relativo e coefficiente R² per modelli di regressione) e analizzando il modello
addestrato (es. coefficienti in una regressione lineare o nodi dei primi livelli di un albero decisionale) per
individuare le variabili più o meno rilevanti nella predizione.)

## Ricerca Iperparametri
(Linee guida: Scegliere uno o più modelli di base (es. regressione ridge) ed eseguire una ricerca degli iperparametri
esaustiva (grid search) o a campione (randomized search) che massimizzi le performance del modello.)